#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m-%d')

#Agora vamos importar nosso arquivo de PIB brasileiro
df = pd.read_csv('..\Data_FS_04.24\PIB.csv')
print(df.head(10))

      PIB        Date
0  189323  1996-03-31
1  204611  1996-06-30
2  221513  1996-09-30
3  239316  1996-12-31
4  219117  1997-03-31
5  232890  1997-06-30
6  246178  1997-09-30
7  253904  1997-12-31
8  235701  1998-03-31
9  251936  1998-06-30


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PIB     112 non-null    int64 
 1   Date    112 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [4]:
#df.loc[:, 'Date'] = pd.to_datetime(df['Date']).date
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   PIB     112 non-null    int64         
 1   Date    112 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.9 KB


### Avaliando os dados historicos do PIB Brasileiro

In [6]:
df.set_index('Date',inplace=True)

In [7]:
df=df.sort_index(axis=0)
print(df.head(10))

               PIB
Date              
1996-03-31  189323
1996-06-30  204611
1996-09-30  221513
1996-12-31  239316
1997-03-31  219117
1997-06-30  232890
1997-09-30  246178
1997-12-31  253904
1998-03-31  235701
1998-06-30  251936


In [8]:
len(df)

112

fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [9]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,PIB
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(112, 1)"
5,Transformed data shape,"(112, 1)"
6,Transformed train set shape,"(111, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [10]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":115})

In [11]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":115})

## Checking Train-Test Split

In [12]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":115})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [13]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [14]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":115})

### xgboost_cds_dt

In [15]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":115})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 29},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":115})
exp.predict_model(xgboost_cds_dt, fh = 29)


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:41:31
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting w/ Cond. Deseasonali...


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2022Q4,0.3542,0.2710,31921.1549,31921.1549,0.0124,0.0123
1,2023Q1,2.2751,1.7396,208790.0910,208790.0910,0.0773,0.0804
2,2023Q2,0.1725,0.1323,15984.3410,15984.3410,0.0058,0.0058
Mean,NaT,0.9339,0.7143,85565.1956,85565.1956,0.0318,0.0328
SD,NaT,0.9512,0.7272,87375.7275,87375.7275,0.0323,0.0337


,y_pred
2023Q4,2.779095e+06
2024Q1,2.747321e+06
2024Q2,2.791446e+06
2024Q3,2.827584e+06
2024Q4,2.865766e+06
2025Q1,2.837747e+06
2025Q2,2.878117e+06
2025Q3,2.914255e+06
2025Q4,2.952437e+06
2026Q1,2.924417e+06


## ets

In [16]:
#ets  = exp.create_model('ets')
#exp.plot_model(plot = 'decomp', 
#               data_kwargs = {'type' : 'multiplicative'},
#               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
#exp.plot_model(estimator = ets, plot = 'forecast', 
#               data_kwargs = {'fh' : 29},
#               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
#exp.predict_model(ets, fh = 29)


#### arima   ARIMA 

In [17]:

arima = exp.create_model('arima')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 29},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(arima, fh = 29)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2022Q4,0.1031,0.0789,9289.1306,9289.1306,0.0036,0.0036
1,2023Q1,0.5657,0.4326,51919.6353,51919.6353,0.0192,0.0190
2,2023Q2,0.2901,0.2225,26872.7038,26872.7038,0.0098,0.0098
Mean,NaT,0.3196,0.2446,29360.4899,29360.4899,0.0109,0.0108
SD,NaT,0.1900,0.1453,17492.5087,17492.5087,0.0064,0.0063


In [ ]:
## AUTO_ARIMA

In [ ]:
auto_arima = exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = auto_arima, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 29},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(auto_arima, fh = 29)

In [ ]:
## stlf
stlf  = exp.create_model('stlf')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = stlf, plot = 'forecast', 
               data_kwargs = {'fh' : 29},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.predict_model(stlf, fh = 29)


## ALL GRAPHS

In [ ]:
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = stlf, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = auto_arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

               

NameError: name 'huber_cds_dt' is not defined